### After the initial EAD, I realized half of the addresses were empty. I have checked socrata version of the data set and found it to be identical to the curated version of the data set. My goal is to populate the missing lattitudes and longtitudes and prior to it, clean up the zipcode list and finally create a neighboorhood column for aggregated analysis.

In [127]:
### Import libraries
# general charting, pandas
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print(pd.__version__) #curious since it is a prebuild image

# To measure progress during apply
# from tqdm import tqdm
# tqdm().pandas()

# For datatime munging
from datetime import date
from datetime import time
from datetime import datetime

# for seaborn charting
import seaborn as sns
sns.set()

# data set socrata
#from sodapy import Socrata

1.0.1


In [128]:
! pip install geopy

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [129]:
# df_location = pd.read_csv('/home/ec2-user/SageMaker/location.csv', sep = ',')  
# df_location.head(3)

missing_values = ["n/a", "na", "--", "unknown", "?", " unknown", " ?"]  # I can add "00000" and "0" here
df = pd.read_csv('/home/ec2-user/SageMaker/scores.csv', sep = ',' ,na_values=missing_values, parse_dates=True)  
df.head(3)

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_type,violation_id,violation_description,risk_category,Neighborhoods (old),Police Districts,Supervisor Districts,Fire Prevention Districts,Zip Codes,Analysis Neighborhoods
0,69618,Fancy Wheatfield Bakery,1362 Stockton St,San Francisco,CA,94133,NaN,NaN,NaN,NaN,...,Complaint,69618_20190304_103130,Inadequate sewage or wastewater disposal,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
1,97975,BREADBELLY,1408 Clement St,San Francisco,CA,94118,NaN,NaN,NaN,1.415724e+10,...,Routine - Unscheduled,97975_20190725_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
2,69487,Hakkasan San Francisco,1 Kearny St,San Francisco,CA,94108,NaN,NaN,NaN,NaN,...,Routine - Unscheduled,69487_20180418_103119,Inadequate and inaccessible handwashing facili...,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
# Socrata connection for future data pipeline work
'''
MyAppToken = '$MyAppToken'
client = Socrata("data.sfgov.org",
                 MyAppToken,
                 username="username@gmail.com",   # issue with username, ceyo does not work must be email
                 password="$password")
# dictionaries by sodapy.
results = client.get("pyih-qa8i", limit= "5500000") # must provide a value even if too large

#Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
'''

'\nMyAppToken = \'$MyAppToken\'\nclient = Socrata("data.sfgov.org",\n                 MyAppToken,\n                 username="username@gmail.com",   # issue with username, ceyo does not work must be email\n                 password="$password")\n# dictionaries by sodapy.\nresults = client.get("pyih-qa8i", limit= "5500000") # must provide a value even if too large\n\n#Convert to pandas DataFrame\nresults_df = pd.DataFrame.from_records(results)\n'

In [131]:
# df_location_distinct = df_location.groupby(['business_id']).max()
# df_location_distinct

df['business_postal_code'] = df['business_postal_code'].str.split('-').str[0]

In [132]:
df_location = pd.DataFrame(df, columns = ['business_id', 'business_address', 'business_city', 'business_state', 'business_postal_code', 'business_latitude', 'business_longitude'])
df_location

,business_id,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude
0,69618,1362 Stockton St,San Francisco,CA,94133,NaN,NaN
1,97975,1408 Clement St,San Francisco,CA,94118,NaN,NaN
2,69487,1 Kearny St,San Francisco,CA,94108,NaN,NaN
3,91044,4615 Mission St,San Francisco,CA,94112,NaN,NaN
4,85987,552 Jones St,San Francisco,CA,94102,NaN,NaN
...,...,...,...,...,...,...,...
53968,89569,2 South Park,San Francisco,CA,94107,NaN,NaN
53969,99764,65 CAMBON Dr,San Francisco,CA,94132,NaN,NaN
53970,84541,300 Folsom St,San Francisco,CA,94105,NaN,NaN
53971,91572,4591 Mission St.,San Francisco,CA,94112,NaN,NaN


In [133]:
#df_location_distinct['business_postal_code'] = df_location_distinct['business_postal_code'].astype(str)          
#df_location_distinct['business_postal_code'] = df_location_distinct['business_postal_code'].str.replace('.0', ' ')

df_location.loc[df_location['business_postal_code'].str.len() != 5.0, ['business_postal_code']] = np.nan
df_location.loc[df_location['business_postal_code'] == '00000', ['business_postal_code']] = np.nan
print(df_location.isna().sum()) # more empty zipcodes, we were able to capture zipcodes that are not 5 digits or 00000
print(df_location.head()) #finally the business_postal_code does not have trailing zeros. 

business_id                 0
business_address            0
business_city               0
business_state              0
business_postal_code     1235
business_latitude       26498
business_longitude      26498
dtype: int64
   business_id  business_address  business_city business_state  \
0        69618  1362 Stockton St  San Francisco             CA   
1        97975   1408 Clement St  San Francisco             CA   
2        69487       1 Kearny St  San Francisco             CA   
3        91044   4615 Mission St  San Francisco             CA   
4        85987      552 Jones St  San Francisco             CA   

  business_postal_code  business_latitude  business_longitude  
0                94133                NaN                 NaN  
1                94118                NaN                 NaN  
2                94108                NaN                 NaN  
3                94112                NaN                 NaN  
4                94102                NaN                 Na

In [134]:
#Proper case for Business_address, hoping to lower distinct business number and make Address field nice
df_location['business_address'] = df_location['business_address'].str.title()
df_location['business_city'] = df_location['business_city'].str.title()

df_location.head()

,business_id,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude
0,69618,1362 Stockton St,San Francisco,CA,94133,NaN,NaN
1,97975,1408 Clement St,San Francisco,CA,94118,NaN,NaN
2,69487,1 Kearny St,San Francisco,CA,94108,NaN,NaN
3,91044,4615 Mission St,San Francisco,CA,94112,NaN,NaN
4,85987,552 Jones St,San Francisco,CA,94102,NaN,NaN


In [135]:
df_location_distinct = df_location.groupby(['business_id']).max()
print(df_location_distinct.shape)
df_location_distinct.head()

# initial I thought I have to run this again after clean up to group by once more but I am grouping by businessid, nothing changes anyway

(6253, 6)


,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude
business_id,,,,,,
19,"1200 Van Ness Ave, 3Rd Floor",San Francisco,CA,94109,37.786848,-122.421547
24,"500 California St, 2Nd Floor",San Francisco,CA,94104,37.792888,-122.403135
31,2801 Leavenworth St,San Francisco,CA,94133,37.807155,-122.419004
45,3202 Folsom St,San Francisco,CA,94110,37.747114,-122.413641
48,747 Irving St,San Francisco,CA,94122,37.764013,-122.465749


In [136]:
# business_addressv2 = []

# for address in df_location_distinct['business_address']:
#     split_string = address.rsplit(",", 1) 
#     substring = split_string[0]
#     business_addressv2.append(substring)
# lat = pd.Series(business_addressv2)
# #df_location_distinct['business_addressv2'] = lat.values

In [137]:
# lat2 = []
# for address in lat:
#     split_string = address.rsplit(".", 1) 
#     substring = split_string[0]
#     lat2.append(substring)
# lat3 = pd.Series(lat2)

In [138]:
# lat4 = []

# for address in lat3:
#     split_string = address.rsplit("#", 1) 
#     substring = split_string[0]
#     lat4.append(substring)
# lat5 = pd.Series(lat4)
# df_location_distinct['business_addressv2'] = lat5.values
# df_location_distinct['business_addressv2'].head()

In [139]:
# ### Remove everything after punctuation

df_location_distinct['business_addressv2'] = df_location_distinct['business_address'].str.replace(r",.*","")
df_location_distinct['business_addressv3'] = df_location_distinct['business_addressv2'].str.replace(r",.*","")
df_location_distinct['business_address_cl'] = df_location_distinct['business_addressv3'].str.replace(r"#.*","")
df_location_distinct.drop(['business_addressv2', 'business_addressv3'], axis = 1, inplace = True) 
df_location_distinct


# I spend extensive time in regex. It is facinating how you can write it according to the ,,, My goal for next year is to do regex without googling.

,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_address_cl
business_id,,,,,,,
19,"1200 Van Ness Ave, 3Rd Floor",San Francisco,CA,94109,37.786848,-122.421547,1200 Van Ness Ave
24,"500 California St, 2Nd Floor",San Francisco,CA,94104,37.792888,-122.403135,500 California St
31,2801 Leavenworth St,San Francisco,CA,94133,37.807155,-122.419004,2801 Leavenworth St
45,3202 Folsom St,San Francisco,CA,94110,37.747114,-122.413641,3202 Folsom St
48,747 Irving St,San Francisco,CA,94122,37.764013,-122.465749,747 Irving St
...,...,...,...,...,...,...,...
102518,611 Post St,San Francisco,CA,94109,NaN,NaN,611 Post St
102577,2470 San Bruno Avenue,San Francisco,CA,94134,NaN,NaN,2470 San Bruno Avenue
102602,202 Clement St,San Francisco,CA,94118,NaN,NaN,202 Clement St


In [140]:
#df_location_distinct['business_addressv2'] = df_location_distinct['business_addressv4'].str.replace(r"\{Way*\}","")

In [141]:
# Load to check
df_location_distinct.to_csv('/home/ec2-user/SageMaker/location_distinct.csv', index=False)

In [142]:
# ### Further cleaning!!! After I received the lat and long I had many no finds, this because of the unit number being added to the address line. I for one will never do this again when I am filling an application. I can see that there is a reason for ther the address line 2. After all these years I finally understand the purpose of the address line 2. I 
# # Everything after the second st.
# a_string = "595 Strack St. Suite 160 St unit 65"
# split_string = a_string.rsplit("St", 1) 
# substring = split_string[0]
# print(substring)

In [143]:
# sentence = 'b a hello b Hi'
# sentence.rsplit('b ', 2)

### Need 2 data sets. One that has lat and long. One that does not have it. Once I fill out the one that does not have it I will merge them

In [144]:
# Drop empty unknown locations - I DONT NEED TO BECAUSE THIS METHOD RETURNS IT.
#df_location_distinct = df_location_distinct.dropna(axis=0, subset=['business_postal_code'])
#201 businesses were dropped sinc ethey do not have a valid zipcode.

# creating the reverse geocode set. These are empty zipcodes with empty lat and long. 
# Feed through the reverse geocode and if that does not work drop it. 

# df_location_reverse_geocode = df_location_distinct.loc[df_location_distinct['business_postal_code'].isnull()].loc[df_location_distinct['business_latitude'].isnull()]
# df_location_reverse_geocode


# Actually dont need it. this finds. Previosu method did not find without a zipcode, this does.

In [145]:
#creating the golden copy of the geocode set. Postal code not null but lat and long are null
#df_location_geocode = df_location_distinct.loc[df_location_distinct['business_postal_code'].notnull()].loc[df_location_distinct['business_latitude'].isnull()]
df_location_geocode = df_location_distinct.loc[df_location_distinct['business_latitude'].isnull()] # geocode
df_location_no_geocode = df_location_distinct.loc[df_location_distinct['business_latitude'].notnull()] # dont geocode
df_location_no_geocode

# From 55K API call to a succint 3348 call. I only need to do this one time too. Plus postal code is correct, lat and long needs to drop though

,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_address_cl
business_id,,,,,,,
19,"1200 Van Ness Ave, 3Rd Floor",San Francisco,CA,94109,37.786848,-122.421547,1200 Van Ness Ave
24,"500 California St, 2Nd Floor",San Francisco,CA,94104,37.792888,-122.403135,500 California St
31,2801 Leavenworth St,San Francisco,CA,94133,37.807155,-122.419004,2801 Leavenworth St
45,3202 Folsom St,San Francisco,CA,94110,37.747114,-122.413641,3202 Folsom St
48,747 Irving St,San Francisco,CA,94122,37.764013,-122.465749,747 Irving St
...,...,...,...,...,...,...,...
71783,"600 Townsend St, 4Th Floor",San Francisco,CA,94103,37.771364,-122.402279,600 Townsend St
71823,200 Pine St #140,San Francisco,CA,94104,37.792237,-122.400124,200 Pine St
71841,2198 Filbert St,San Francisco,CA,94123,37.798078,-122.435444,2198 Filbert St


In [146]:
# drop lat and long and org business address. I will use cln version
df_location_geocode_cln=df_location_geocode.drop(columns =['business_latitude', 'business_longitude', 'business_address'])
df_location_geocode_cln.head()

,business_city,business_state,business_postal_code,business_address_cl
business_id,,,,
852,San Francisco,CA,94133,652 Pacific Ave
2934,San Francisco,CA,94133,300 Jefferson St
4323,San Francisco,CA,94109,1081 Post St
5281,San Francisco,CA,94122,Pier 45
5312,San Francisco,CA,94111,Pier 9


In [147]:
df_location_geocode_cln['address'] = df_location_geocode_cln['business_address_cl'].astype(str) + ' ' + df_location_geocode_cln['business_city'].astype(str) + ' '  + df_location_geocode_cln['business_state'].astype(str) + ' ' + df_location_geocode_cln['business_postal_code'].astype(str)
df_location_geocode_cln.head()

#This method is better then Open GI because I dont have to use the comma between state and zipcode. Another time saver.

,business_city,business_state,business_postal_code,business_address_cl,address
business_id,,,,,
852,San Francisco,CA,94133,652 Pacific Ave,652 Pacific Ave San Francisco CA 94133
2934,San Francisco,CA,94133,300 Jefferson St,300 Jefferson St San Francisco CA 94133
4323,San Francisco,CA,94109,1081 Post St,1081 Post St San Francisco CA 94109
5281,San Francisco,CA,94122,Pier 45,Pier 45 San Francisco CA 94122
5312,San Francisco,CA,94111,Pier 9,Pier 9 San Francisco CA 94111


In [148]:
# Add but no need anymore

#Create a test set - Always a good idea to check the speed.
df_location_geocode_test = df_location_geocode_cln[:25]
df_location_geocode_test.head()

,business_city,business_state,business_postal_code,business_address_cl,address
business_id,,,,,
852,San Francisco,CA,94133,652 Pacific Ave,652 Pacific Ave San Francisco CA 94133
2934,San Francisco,CA,94133,300 Jefferson St,300 Jefferson St San Francisco CA 94133
4323,San Francisco,CA,94109,1081 Post St,1081 Post St San Francisco CA 94109
5281,San Francisco,CA,94122,Pier 45,Pier 45 San Francisco CA 94122
5312,San Francisco,CA,94111,Pier 9,Pier 9 San Francisco CA 94111


In [149]:
df_location_geocode_cln.shape

(3497, 5)

In [150]:
3497/2

1748.5

In [151]:
df_location_geocode_cln1=df_location_geocode_cln.iloc[:1745]
df_location_geocode_cln2=df_location_geocode_cln.iloc[1746:]
#This allowed me to do it in parallel

### Run all above for reset

In [152]:
len(df_location_geocode_cln)

3497

In [153]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim   # NOMINATIM USE OPEN STREET MAP DATA
geolocator = Nominatim(user_agent="scoopv2")

# function to delay between geocoding calls. I read you can get kicked out after ~30mins for inactivity
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# create location column for the API call.
df_location_geocode_cln['location'] = df_location_geocode_cln['address'].apply(geocode)

# create longitude, latitude and altitude from location column (returns tuple), didnt want to drop altitude at this point
df_location_geocode_cln['point'] = df_location_geocode_cln['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# split point column into latitude, longitude and altitude columns
df_location_geocode_cln[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_location_geocode_cln['point'].tolist(), index=df_location_geocode_cln.index)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('711  San Francisco CA 94103',), **{}).
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/http/client.py", line 1354, in getresponse
    response.begin()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/home/ec2-user/anaconda3/envs/python3/lib/python

In [154]:
df_location_geocode_cln.head()

,business_city,business_state,business_postal_code,business_address_cl,address,location,point,latitude,longitude,altitude
business_id,,,,,,,,,,
852,San Francisco,CA,94133,652 Pacific Ave,652 Pacific Ave San Francisco CA 94133,"(652, Pacific Avenue, Chinatown, San Francisco...","(37.797083428571426, -122.4060566122449, 0.0)",37.797083,-122.406057,0.0
2934,San Francisco,CA,94133,300 Jefferson St,300 Jefferson St San Francisco CA 94133,"(Lou's Fish Shack, 300, Jefferson Street, Russ...","(37.8081258, -122.41790142012, 0.0)",37.808126,-122.417901,0.0
4323,San Francisco,CA,94109,1081 Post St,1081 Post St San Francisco CA 94109,"(1081, Post Street, Tenderloin, San Francisco,...","(37.78662695, -122.41949325338396, 0.0)",37.786627,-122.419493,0.0
5281,San Francisco,CA,94122,Pier 45,Pier 45 San Francisco CA 94122,"(Pier 45, Russian Hill, San Francisco, San Fra...","(37.8101189, -122.4177162, 0.0)",37.810119,-122.417716,0.0
5312,San Francisco,CA,94111,Pier 9,Pier 9 San Francisco CA 94111,"(Pier 9, The Embarcadero, Northeast Waterfront...","(37.800791399999994, -122.3965530928354, 0.0)",37.800791,-122.396553,0.0


In [155]:
len(df_location_geocode_cln)

3497

### Further cleaning

In [157]:
#Back up first
df_location_geocode_cln.to_csv('/home/ec2-user/SageMaker/df_location_geocode_cln.csv', index=True)

# And within the hour I needed the back. So thankful not to loose another hour for processing

In [158]:
df_location_geocode_cln = pd.read_csv('/home/ec2-user/SageMaker/df_location_geocode_cln.csv', sep = ',')
df_location_geocode_cln.head(3)

,business_id,business_city,business_state,business_postal_code,business_address_cl,address,location,point,latitude,longitude,altitude
0,852,San Francisco,CA,94133.0,652 Pacific Ave,652 Pacific Ave San Francisco CA 94133,"652, Pacific Avenue, Chinatown, San Francisco,...","(37.797083428571426, -122.4060566122449, 0.0)",37.797083,-122.406057,0.0
1,2934,San Francisco,CA,94133.0,300 Jefferson St,300 Jefferson St San Francisco CA 94133,"Lou's Fish Shack, 300, Jefferson Street, Russi...","(37.8081258, -122.41790142012, 0.0)",37.808126,-122.417901,0.0
2,4323,San Francisco,CA,94109.0,1081 Post St,1081 Post St San Francisco CA 94109,"1081, Post Street, Tenderloin, San Francisco, ...","(37.78662695, -122.41949325338396, 0.0)",37.786627,-122.419493,0.0


In [159]:
### Prepare the df_location_no_geocode dataset. Rearrange columns and rename.
cols= ['business_address_cl', 'business_city', 'business_state', 'business_postal_code', 'latitude', 'longitude']
df_location_geocode = df_location_geocode_cln[cols]
df_location_geocode = df_location_clean.rename(columns={"latitude": "business_latitude", "longitude" : "business_longitude", "business_address_cl":"business_address"})
df_location_geocode.head(3)

,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude
business_id,,,,,,
852,652 Pacific Ave,San Francisco,CA,94133,37.797083,-122.406057
2934,300 Jefferson St,San Francisco,CA,94133,37.808126,-122.417901
4323,1081 Post St,San Francisco,CA,94109,37.786627,-122.419493


In [160]:
### Prepare the df_location_no_geocode dataset. Rearrange columns and rename.
cols= ['business_address_cl', 'business_city', 'business_state', 'business_postal_code', 'business_latitude', 'business_longitude']
df_location_nogeocode = df_location_no_geocode[cols]
df_location_nogeocode = df_location_nogeocode.rename(columns={"business_address_cl": "business_address"})
df_location_nogeocode.head(3)

,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude
business_id,,,,,,
19,1200 Van Ness Ave,San Francisco,CA,94109,37.786848,-122.421547
24,500 California St,San Francisco,CA,94104,37.792888,-122.403135
31,2801 Leavenworth St,San Francisco,CA,94133,37.807155,-122.419004


In [161]:
# concat with two datasets
frames = [df_location_geocode, df_location_nogeocode]

df_location_master = pd.concat(frames)
df_location_master.shape

(6253, 6)

In [162]:
df_location_master = df_location_master.loc[df_location_master['business_latitude'].notnull()]
df_location_master.shape

(5448, 6)

In [163]:
df_location_master.head(7)

,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude
business_id,,,,,,
852,652 Pacific Ave,San Francisco,CA,94133,37.797083,-122.406057
2934,300 Jefferson St,San Francisco,CA,94133,37.808126,-122.417901
4323,1081 Post St,San Francisco,CA,94109,37.786627,-122.419493
5281,Pier 45,San Francisco,CA,94122,37.810119,-122.417716
5312,Pier 9,San Francisco,CA,94111,37.800791,-122.396553
5313,Pier 41,San Francisco,CA,94133,37.809474,-122.411851
5409,1 United Nations Plaza,San Francisco,CA,94102,37.780161,-122.413779


In [164]:
df_location_master.to_csv('/home/ec2-user/SageMaker/df_location_master.csv', index=True)

# Create neighboorhood field

In [165]:
equiv = {"94102":"Hayes Valley/Tenderloin/North of Market", "94103":"South of Market", "94107":"Potrero Hill", "94108":"Chinatown"
, "94109":"Polk/Russian Hill (Nob Hill)", "94110":"Inner Mission/Bernal Heights", "94112":"Ingelside-Excelsior/Crocker-Amazon", "94114":"Castro/Noe Valley"
, "94115":"Western Addition/Japantown", "94116":"Parkside/Forest Hill", "94117":"Haight-Ashbury", "94118":"Inner Richmond", "94121":"Outer Richmond"
, "94122":"Sunset", "94123":"Marina", "94124":"Bayview-Hunters Point", "94127":"St. Francis Wood/Miraloma/West Portal", "94131":"Twin Peaks-Glen Park"
, "94132":"Lake Merced", "94133":"North Beach/Chinatown", "94134":"Visitacion Valley/Sunnydale", "94111":"75 Howard Area", "94104":"NoName-94104", "94105":"East Cut"}

df_location_master["neighborhood"] = df_location_master["business_postal_code"].map(equiv)

In [166]:
df_location_master.head(7)

,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,neighborhood
business_id,,,,,,,
852,652 Pacific Ave,San Francisco,CA,94133,37.797083,-122.406057,North Beach/Chinatown
2934,300 Jefferson St,San Francisco,CA,94133,37.808126,-122.417901,North Beach/Chinatown
4323,1081 Post St,San Francisco,CA,94109,37.786627,-122.419493,Polk/Russian Hill (Nob Hill)
5281,Pier 45,San Francisco,CA,94122,37.810119,-122.417716,Sunset
5312,Pier 9,San Francisco,CA,94111,37.800791,-122.396553,75 Howard Area
5313,Pier 41,San Francisco,CA,94133,37.809474,-122.411851,North Beach/Chinatown
5409,1 United Nations Plaza,San Francisco,CA,94102,37.780161,-122.413779,Hayes Valley/Tenderloin/North of Market


In [167]:
df_location_master[df_location_master['neighborhood'].isnull()]

,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,neighborhood
business_id,,,,,,,
74485,455 South Mission Bay Blvd,San Francisco,CA,94158,37.770587,-122.391075,NaN
74674,101 Bayshore Blvd,San Francisco,CA,94014,37.746822,-122.403348,NaN
79607,Off The Grid,San Francisco,CA,94129,37.805890,-122.431720,NaN
79829,401 13Th St,San Francisco,CA,94130,37.828373,-122.371981,NaN
80554,530 Parnassus Ave,San Francisco,CA,94143,37.763236,-122.459005,NaN
...,...,...,...,...,...,...,...
69012,2826 Jones St,San Francisco,CA,NaN,37.807673,-122.417266,NaN
69066,550 Gough St,San Francisco,CA,NaN,37.778299,-122.423187,NaN
69122,Off The Grid,San Francisco,CA,NaN,0.000000,0.000000,NaN


In [168]:
df_location_masterv2=df_location_master[df_location_master['business_postal_code'].notnull()]

In [169]:
df_location_masterv2.tail(10)

,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,neighborhood
business_id,,,,,,,
71510,222 Mason St 2Nd Foor,San Francisco,CA,94102,37.785556,-122.409438,Hayes Valley/Tenderloin/North of Market
71532,6901 Geary Blvd,San Francisco,CA,94121,37.779663,-122.493618,Outer Richmond
71563,810 Mission St,San Francisco,CA,94103,37.784193,-122.404699,South of Market
71657,534 Irving St,San Francisco,CA,94122,37.764188,-122.463683,Sunset
71696,2821 California St,San Francisco,CA,94115,37.788014,-122.440756,Western Addition/Japantown
71783,600 Townsend St,San Francisco,CA,94103,37.771364,-122.402279,South of Market
71823,200 Pine St,San Francisco,CA,94104,37.792237,-122.400124,NoName-94104
71841,2198 Filbert St,San Francisco,CA,94123,37.798078,-122.435444,Marina
71909,696 Monterey Blvd,San Francisco,CA,94127,37.731568,-122.450970,St. Francis Wood/Miraloma/West Portal


In [170]:
df_location_masterv2.to_csv('/home/ec2-user/SageMaker/df_location_masterv2.csv', index=True)

# Create a csv for back up.

In [171]:
df.columns


Index(['business_id', 'business_name', 'business_address', 'business_city',
       'business_state', 'business_postal_code', 'business_latitude',
       'business_longitude', 'business_location', 'business_phone_number',
       'inspection_id', 'inspection_date', 'inspection_score',
       'inspection_type', 'violation_id', 'violation_description',
       'risk_category', 'Neighborhoods (old)', 'Police Districts',
       'Supervisor Districts', 'Fire Prevention Districts', 'Zip Codes',
       'Analysis Neighborhoods'],
      dtype='object')

In [172]:
df_updated=df.drop(columns =['business_address','business_city','business_state','business_postal_code','business_latitude',
       'business_longitude', 'business_location', 'business_phone_number', 'Neighborhoods (old)', 'Police Districts',
       'Supervisor Districts', 'Fire Prevention Districts', 'Zip Codes',
       'Analysis Neighborhoods'])
df_updated

,business_id,business_name,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,69618,Fancy Wheatfield Bakery,69618_20190304,03/04/2019 12:00:00 AM,NaN,Complaint,69618_20190304_103130,Inadequate sewage or wastewater disposal,Moderate Risk
1,97975,BREADBELLY,97975_20190725,07/25/2019 12:00:00 AM,96.0,Routine - Unscheduled,97975_20190725_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk
2,69487,Hakkasan San Francisco,69487_20180418,04/18/2018 12:00:00 AM,88.0,Routine - Unscheduled,69487_20180418_103119,Inadequate and inaccessible handwashing facili...,Moderate Risk
3,91044,Chopsticks Restaurant,91044_20170818,08/18/2017 12:00:00 AM,NaN,Non-inspection site visit,NaN,NaN,NaN
4,85987,Tselogs,85987_20180412,04/12/2018 12:00:00 AM,94.0,Routine - Unscheduled,85987_20180412_103132,Improper thawing methods,Moderate Risk
...,...,...,...,...,...,...,...,...,...
53968,89569,Blue Bottle Coffee,89569_20190506,05/06/2019 12:00:00 AM,80.0,Routine - Unscheduled,89569_20190506_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk
53969,99764,POKE KANA,99764_20190314,03/14/2019 12:00:00 AM,NaN,New Ownership - Followup,NaN,NaN,NaN
53970,84541,Philz Coffee,84541_20190506,05/06/2019 12:00:00 AM,92.0,Routine - Unscheduled,84541_20190506_103133,Foods not protected from contamination,Moderate Risk
53971,91572,El Gran Taco Loco,91572_20190506,05/06/2019 12:00:00 AM,76.0,Routine - Unscheduled,91572_20190506_103116,Inadequate food safety knowledge or lack of ce...,Moderate Risk


In [173]:
df_location_masterv2

,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,neighborhood
business_id,,,,,,,
852,652 Pacific Ave,San Francisco,CA,94133,37.797083,-122.406057,North Beach/Chinatown
2934,300 Jefferson St,San Francisco,CA,94133,37.808126,-122.417901,North Beach/Chinatown
4323,1081 Post St,San Francisco,CA,94109,37.786627,-122.419493,Polk/Russian Hill (Nob Hill)
5281,Pier 45,San Francisco,CA,94122,37.810119,-122.417716,Sunset
5312,Pier 9,San Francisco,CA,94111,37.800791,-122.396553,75 Howard Area
...,...,...,...,...,...,...,...
71783,600 Townsend St,San Francisco,CA,94103,37.771364,-122.402279,South of Market
71823,200 Pine St,San Francisco,CA,94104,37.792237,-122.400124,NoName-94104
71841,2198 Filbert St,San Francisco,CA,94123,37.798078,-122.435444,Marina


### I have 2 sets and I want to inner merge them

In [175]:
inner_merged_total = pd.merge(df_location_masterv2,df_updated, on=["business_id"])
inner_merged_total.head()
inner_merged_total.shape


(48765, 16)

In [176]:
inner_merged_total.head(3)

,business_id,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,neighborhood,business_name,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,852,652 Pacific Ave,San Francisco,CA,94133,37.797083,-122.406057,North Beach/Chinatown,New Hollywood Bakery & Restaurant,852_20171129,11/29/2017 12:00:00 AM,83.0,Routine - Unscheduled,852_20171129_103139,Improper food storage,Low Risk
1,852,652 Pacific Ave,San Francisco,CA,94133,37.797083,-122.406057,North Beach/Chinatown,New Hollywood Bakery & Restaurant,852_20171129,11/29/2017 12:00:00 AM,83.0,Routine - Unscheduled,852_20171129_103156,Permit license or inspection report not posted,Low Risk
2,852,652 Pacific Ave,San Francisco,CA,94133,37.797083,-122.406057,North Beach/Chinatown,New Hollywood Bakery & Restaurant,852_20171129,11/29/2017 12:00:00 AM,83.0,Routine - Unscheduled,852_20171129_103144,Unapproved or unmaintained equipment or utensils,Low Risk


In [177]:
inner_merged_total.columns

Index(['business_id', 'business_address', 'business_city', 'business_state',
       'business_postal_code', 'business_latitude', 'business_longitude',
       'neighborhood', 'business_name', 'inspection_id', 'inspection_date',
       'inspection_score', 'inspection_type', 'violation_id',
       'violation_description', 'risk_category'],
      dtype='object')

In [178]:
cols= ['business_id', 'business_name', 'business_address', 'business_city', 'business_state',
       'business_postal_code', 'business_latitude', 'business_longitude',
       'neighborhood', 'inspection_id', 'inspection_date',
       'inspection_score', 'inspection_type', 'violation_id',
       'violation_description', 'risk_category']
df_location_master_final = inner_merged_total[cols]
df_location_master_final.columns

Index(['business_id', 'business_name', 'business_address', 'business_city',
       'business_state', 'business_postal_code', 'business_latitude',
       'business_longitude', 'neighborhood', 'inspection_id',
       'inspection_date', 'inspection_score', 'inspection_type',
       'violation_id', 'violation_description', 'risk_category'],
      dtype='object')

In [179]:
df_location_master_final.to_csv('/home/ec2-user/SageMaker/df_location_master_final.csv', index=True)

# Final data set exported. Next, it will be moved to R Studio Cloud for further analysis

# Appendix #############

In [180]:
## Geocode - Original code. referenced from GeoPy Pypi.
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="scoopv2")
location = geolocator.geocode("1200 Van Ness Ave San Francisco CA")
print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

24 Hour Fitness, 1200, Van Ness Avenue, Tenderloin, San Francisco, San Francisco City and County, California, 94164, United States of America
(37.7867992, -122.4212723)
{'place_id': 3783745, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 417957202, 'boundingbox': ['37.7867492', '37.7868492', '-122.4213223', '-122.4212223'], 'lat': '37.7867992', 'lon': '-122.4212723', 'display_name': '24 Hour Fitness, 1200, Van Ness Avenue, Tenderloin, San Francisco, San Francisco City and County, California, 94164, United States of America', 'class': 'leisure', 'type': 'fitness_centre', 'importance': 0.711}


## Reverse Geocode

In [181]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)

print((location.latitude, location.longitude))

print(location.raw)

ConfigurationError: Using Nominatim with default or sample `user_agent` "specify_your_app_name_here" is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`.

## Measure distance (Future work - maybe a distance feature)

In [ ]:
from geopy.distance import geodesic
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(geodesic(newport_ri, cleveland_oh).miles)

In [ ]:
#Department of Public Health - 101 Grove St, San Francisco, CA 94102 - In case I need to measure the distance

In [ ]:
# import pandas as pd
# equiv = {7001:1, 8001:2, 9001:3}
# df = pd.DataFrame( {"A": [7001, 8001, 9001, 10000]} )
# df["B"] = df["A"].map(equiv)
# print(df)